In [ ]:
import os
import time
import math
import re
import struct
import traceback
import codecs

import bitstring

import numpy as np
import matplotlib.pyplot as plt
import cv2

from PIL import Image
import IPython.display as display

In [ ]:
os.listdir('.')

In [ ]:
file_list = [
    'ETL8G_01',
    'ETL8G_02',
    'ETL8G_03',
    'ETL8G_04',
    'ETL8G_05',
    'ETL8G_06',
    'ETL8G_07',
    'ETL8G_08',
    'ETL8G_09',
    'ETL8G_10',
    'ETL8G_11',
    'ETL8G_12',
    'ETL8G_13',
    'ETL8G_14',
    'ETL8G_15',
    'ETL8G_16',
    'ETL8G_17',
    'ETL8G_18',
    'ETL8G_19',
    'ETL8G_20',
    'ETL8G_21',
    'ETL8G_22',
    'ETL8G_23',
    'ETL8G_24',
    'ETL8G_25',
    'ETL8G_26',
    'ETL8G_27',
    'ETL8G_28',
    'ETL8G_29',
    'ETL8G_30',
    'ETL8G_31',
    'ETL8G_32',
    'ETL8G_33',
]

In [ ]:
filename = file_list[0]
print(filename)
file_stream = bitstring.ConstBitStream(filename=filename)

In [ ]:
record = file_stream.readlist(','.join([
    'int:16', # Serial Sheet Number - [0]
    'uint:16', # JIS Character Code (JIS X 0208) - [1]
    '8*uint:8', # JIS Typical Reading (e.g. 'AI.MEDER') - [2:10]
    'int:32', # Serial Data Number - [10]
    '4*int:8', # - [11:15]
    '4*uint:16', # - [15:19]
    '2*int:8', # x and y - [19:21]
    'pad:240',
    'bytes:8128', # 16-graylevel (4-bit) image of 128 x 127 = 16256 pixels
    'pad:88',
]))

In [ ]:
print(type(record), len(record))
for idx, v in enumerate(record[:-1]):
    print(f'{idx} - {v}')

In [ ]:
record_dict = {
    'JIS_CODE': record[1],
    'image_data': record[-1]
}

In [ ]:
hex(record_dict['JIS_CODE'])

In [ ]:
width = 128
height = 127
img = np.array(Image.frombytes('F', (width, height), record_dict['image_data'], 'bit', 4))
plt.imshow(img)
plt.colorbar()

In [ ]:
total_samples = 0
record_count = {}

for filename in file_list:
    file_stream = bitstring.ConstBitStream(filename=filename)

    while True:
        try:
            record = file_stream.readlist(','.join([
                'int:16', # Serial Sheet Number - [0]
                'uint:16', # JIS Character Code (JIS X 0208) - [1]
                '8*uint:8', # JIS Typical Reading (e.g. 'AI.MEDER') - [2:10]
                'int:32', # Serial Data Number - [10]
                '4*int:8', # - [11:15]
                '4*uint:16', # - [15:19]
                '2*int:8', # x and y - [19:21]
                'pad:240',
                'bytes:8128', # 16-graylevel (4-bit) image of 128 x 127 = 16256 pixels
                'pad:88',
            ]))
        except:
            break
        
        total_samples += 1
        record_dict = {
            'JIS_CODE': record[1],
            'image_data': record[-1],
        }
        
        name = hex(record_dict['JIS_CODE'])
        if name in record_count.keys():
            record_count[name] += 1
        else:
            record_count[name] = 1

In [ ]:
total_samples

In [ ]:
len(record_count.keys())

In [ ]:
record_count